In [1]:
#Para obtener el csv de las estaciones de carga de EE.UU. con la información que necesitamos
import pandas as pd
import numpy as np

df = pd.read_csv('eeuu_fuel_stations.csv', low_memory=False)
df = df.sample(frac=1, random_state=np.random.RandomState())

df2 = df[[ "Station Name", "Latitude", "Longitude", "EV Level2 EVSE Num"]]
#Eliminamos los NULL
df2 = df2[df["EV Level2 EVSE Num"] > 0]
df2["Station Name"].astype(str)
df2.to_csv('eeuu_fuel_stations_processed.csv', index=False)
df2


,Station Name,Latitude,Longitude,EV Level2 EVSE Num
25094,DELTA AIR LINES TOC EV CHARGER1,33.641652,-84.415299,2.0
59545,275 west Hunt Club Rd - 2,45.340513,-75.709017,4.0
2009,City of Pasadena - Marengo Garage,34.144703,-118.146957,5.0
26497,BURNS MGT THE SPRINGS #3,43.093051,-73.743350,2.0
37448,Guarenteed Rate Field Lot D,41.830674,-87.636027,2.0
...,...,...,...,...
47470,Hotel Monaco Seattle,47.607141,-122.333917,1.0
49136,Terrace Place Garage,36.150209,-86.799708,2.0
60217,Karl of Marshalltown,42.062840,-92.908886,1.0
16128,Kohl's at Rolling Hills Plaza,33.793911,-118.331992,2.0


In [2]:
df2["index"] = df2.index + 1
df2.head()

,Station Name,Latitude,Longitude,EV Level2 EVSE Num,index
54650,SUMMIT TOYOTA STATION 1,41.063476,-81.577133,1.0,54651
23511,Two Wisconsin Circle - Tesla Destination,38.961970,-77.085332,1.0,23512
41484,Middle Georgia State University - Cochran,32.380615,-83.346060,2.0,41485
49427,DeForest Athletic Complex,43.221461,-89.362470,4.0,49428
3059,STOWE ELECTRIC REC PATH,44.466002,-72.684932,2.0,3060


In [1]:
#Para importar la información a DynamoDB

#Importacion de las electrolineras
import boto3
from decimal import Decimal
import json

dynamodb = boto3.resource('dynamodb', region_name='us-east-1')
electrolineras_tabla = dynamodb.Table('Electrolinera')

#Eliminamos las entradas anteriores
response = electrolineras_tabla.scan()
for i in response['Items']:
    electrolineras_tabla.delete_item(
        Key={
            'ID_Electrolinera': i['ID_Electrolinera']
        }
    )

def import_electrolinera(row):
    item = {
            'ID_Electrolinera': row["index"],
            'nombre': row['Station Name'],
            'latitud': row['Latitude'],
            'longitud': row['Longitude'],
            'estaciones': row['EV Level2 EVSE Num'],
            'observaciones': "",
            'estado': 1
        }

    electrolineras_tabla.put_item(
        Item = json.loads(json.dumps(item), parse_float=Decimal)
    )

df = pd.read_csv('eeuu_fuel_stations_processed.csv', low_memory=False)
df_mini = df.head(10)
df_mini.apply(import_electrolinera, axis=1)

ClientError: An error occurred (ExpiredTokenException) when calling the Scan operation: The security token included in the request is expired

In [4]:
#Ahora generamos un punto de carga por cada electrolinera importada
df_mini["puntos_de_carga"] = df_mini["EV Level2 EVSE Num"].apply(lambda x: [i for i in range(1, int(x)+1)])
df_mini = df_mini.explode("puntos_de_carga")
df_mini


/tmp/ipykernel_8275/1887988630.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_mini["puntos_de_carga"] = df_mini["EV Level2 EVSE Num"].apply(lambda x: [i for i in range(1, int(x)+1)])


,Station Name,Latitude,Longitude,EV Level2 EVSE Num,index,puntos_de_carga
54650,SUMMIT TOYOTA STATION 1,41.063476,-81.577133,1.0,54651,1
23511,Two Wisconsin Circle - Tesla Destination,38.961970,-77.085332,1.0,23512,1
41484,Middle Georgia State University - Cochran,32.380615,-83.346060,2.0,41485,1
41484,Middle Georgia State University - Cochran,32.380615,-83.346060,2.0,41485,2
49427,DeForest Athletic Complex,43.221461,-89.362470,4.0,49428,1
49427,DeForest Athletic Complex,43.221461,-89.362470,4.0,49428,2
49427,DeForest Athletic Complex,43.221461,-89.362470,4.0,49428,3
49427,DeForest Athletic Complex,43.221461,-89.362470,4.0,49428,4
3059,STOWE ELECTRIC REC PATH,44.466002,-72.684932,2.0,3060,1
3059,STOWE ELECTRIC REC PATH,44.466002,-72.684932,2.0,3060,2


In [5]:

#Importacion de los puntos de carga
puntos_de_carga_tabla = dynamodb.Table('PuntoCarga')

#Eliminamos las entradas anteriores
response = puntos_de_carga_tabla.scan()
for i in response['Items']:
    puntos_de_carga_tabla.delete_item(
        Key={
            'ID_PuntoCarga': i['ID_PuntoCarga']
        }
    )

def import_punto_de_carga(row):
        item = {
                'ID_PuntoCarga': row["index"]*1000 + row["puntos_de_carga"],
                'ID_Electrolinera': row["index"],
                'num_pc_electrolinera': row['puntos_de_carga'],
                'estado': 1,
                'uptime': 0
            }
    
        puntos_de_carga_tabla.put_item(
            Item = json.loads(json.dumps(item), parse_float=Decimal)
        )

df_mini.apply(import_punto_de_carga, axis=1)

54650    None
23511    None
41484    None
41484    None
49427    None
49427    None
49427    None
49427    None
3059     None
3059     None
52487    None
52487    None
19009    None
19009    None
20035    None
20035    None
2055     None
2055     None
2055     None
11804    None
11804    None
dtype: object